In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv(r'C:\Users\RUSHI\Desktop\b daily practice\project\rest review\review.tsv', delimiter='\t', quoting=3)

In [3]:
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [4]:
data.shape

(1000, 2)

In [5]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RUSHI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
corpus=[]
for i in range (0,1000):
    review=re.sub(pattern='[^a-zA-Z]',repl=' ',string=data['Review'][i])
    review=review.lower()
    review_words=review.split()
    review_words=[word for word in review_words if not word in set(stopwords.words('english'))]
    
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review_words]
    
    review=' '.join(review)
    
    corpus.append(review)

In [7]:
corpus[0:5]

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
x=cv.fit_transform(corpus).toarray()
y=data.iloc[:,1].values

In [9]:
import pickle
pickle.dump(cv, open('cv-transform.pkl', 'wb'))

In [10]:
from sklearn.model_selection import train_test_split
xtr,xts,ytr,yts=train_test_split(x,y,test_size=0.2)

In [11]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
classifier.fit(xtr,ytr)
ypre=classifier.predict(xts)

In [12]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [13]:
accuracy_score(ypre,yts)

0.805

In [14]:
confusion_matrix(yts,ypre)

array([[75, 26],
       [13, 86]], dtype=int64)

In [15]:
precision_score(yts,ypre)

0.7678571428571429

In [16]:
# Hyperparameter tuning the Naive Bayes Classifier
best_accuracy = 0.0
alpha_val = 0.0
for i in np.arange(0.1,1.1,0.1):
  temp_classifier = MultinomialNB(alpha=i)
  temp_classifier.fit(xtr, ytr)
  temp_y_pred = temp_classifier.predict(xts)
  score = accuracy_score(yts, temp_y_pred)
  print("Accuracy score for alpha={} is: {}%".format(round(i,1), round(score*100,2)))
  if score>best_accuracy:
    best_accuracy = score
    alpha_val = i
print('--------------------------------------------')
print('The best accuracy is {}% with alpha value as {}'.format(round(best_accuracy*100, 2), round(alpha_val,1)))

Accuracy score for alpha=0.1 is: 79.0%
Accuracy score for alpha=0.2 is: 79.5%
Accuracy score for alpha=0.3 is: 80.0%
Accuracy score for alpha=0.4 is: 80.5%
Accuracy score for alpha=0.5 is: 80.5%
Accuracy score for alpha=0.6 is: 80.5%
Accuracy score for alpha=0.7 is: 80.5%
Accuracy score for alpha=0.8 is: 80.5%
Accuracy score for alpha=0.9 is: 80.5%
Accuracy score for alpha=1.0 is: 80.5%
--------------------------------------------
The best accuracy is 80.5% with alpha value as 0.4


In [17]:

classifier = MultinomialNB(alpha=0.2)
classifier.fit(xtr, ytr)

MultinomialNB(alpha=0.2)

In [18]:
filename = 'restaurant-sentiment.pkl'
pickle.dump(classifier, open(filename, 'wb'))